<a href="https://colab.research.google.com/github/os-1693/hugging-face/blob/main/notebooks/inference_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# モデル推論

学習済みモデルを使用して推論を行います。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

## 1. 環境設定

まず、GPUが有効になっているか確認します。

**設定方法**: メニュー → ランタイム → ランタイムのタイプを変更 → GPU を選択

In [7]:
# GPU確認
import torch
print(f"GPU利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")

GPU利用可能: True
GPU名: Tesla T4


## 2. ライブラリのインストール

In [8]:
!pip install -q transformers

## 3. ライブラリのインポート

In [9]:
from typing import List, Union, Optional, Dict, Any
from transformers import (
    PreTrainedModel,
    PreTrainedTokenizer,
    pipeline,
    Pipeline
)
import torch
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 4. ModelInferenceクラスの定義

In [10]:
class ModelInference:
    """
    モデル推論クラス

    学習済みモデルを使用して推論を行います。
    テキスト分類、トークン分類、質問応答、テキスト生成に対応。

    Attributes:
        model (PreTrainedModel): 推論対象のモデル
        tokenizer (PreTrainedTokenizer): トークナイザー
        device (str): 推論デバイス（'cpu' or 'cuda'）
        task (str): タスクタイプ
        pipeline (Pipeline): Hugging Face pipeline
    """

    def __init__(
        self,
        model: Optional[PreTrainedModel] = None,
        tokenizer: Optional[PreTrainedTokenizer] = None,
        model_path: Optional[str] = None,
        task: str = "text-classification",
        device: Optional[str] = None
    ):
        """
        Args:
            model: 推論対象のモデル
            tokenizer: トークナイザー
            model_path: モデルパス（modelがNoneの場合）
            task: タスクタイプ
            device: 推論デバイス
        """
        self.task = task

        # デバイスの設定
        if device is None:
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device

        logger.info(f"Using device: {self.device}")

        # パイプラインの作成
        if model_path:
            self.pipeline = pipeline(
                task=task,
                model=model_path,
                tokenizer=model_path,
                device=0 if self.device == "cuda" else -1
            )
            self.model = self.pipeline.model
            self.tokenizer = self.pipeline.tokenizer
        elif model and tokenizer:
            self.model = model.to(self.device)
            self.tokenizer = tokenizer
            self.pipeline = pipeline(
                task=task,
                model=model,
                tokenizer=tokenizer,
                device=0 if self.device == "cuda" else -1
            )
        else:
            raise ValueError("model_path または modelとtokenizerの両方を指定してください")

        logger.info(f"Model loaded for task: {task}")

    def predict(
        self,
        inputs: Union[str, List[str]],
        **kwargs
    ) -> Union[Dict[str, Any], List[Dict[str, Any]]]:
        """
        推論を実行

        Args:
            inputs: 入力テキストまたはテキストのリスト
            **kwargs: パイプライン固有の引数

        Returns:
            推論結果
        """
        logger.info(f"Running inference on {len(inputs) if isinstance(inputs, list) else 1} input(s)")

        # デフォルトのkwargs設定
        default_kwargs = {}
        if self.task == "text-classification":
            default_kwargs.update({"return_all_scores": True})
        elif self.task == "text-generation":
            default_kwargs.update({
                "max_new_tokens": 50,
                "num_return_sequences": 1,
                "temperature": 0.7,
                "truncation": True
            })
        elif self.task == "question-answering":
            # QAの場合はinputsがdictであることを期待
            pass

        # kwargsのマージ
        default_kwargs.update(kwargs)

        try:
            results = self.pipeline(inputs, **default_kwargs)
            logger.info("Inference completed successfully")
            return results
        except Exception as e:
            logger.error(f"Inference failed: {e}")
            raise

    def predict_batch(
        self,
        inputs: List[str],
        batch_size: int = 32,
        **kwargs
    ) -> List[Dict[str, Any]]:
        """
        バッチ推論を実行

        Args:
            inputs: 入力テキストのリスト
            batch_size: バッチサイズ
            **kwargs: パイプライン固有の引数

        Returns:
            推論結果のリスト
        """
        logger.info(f"Running batch inference with batch_size={batch_size}")

        results = []
        for i in range(0, len(inputs), batch_size):
            batch = inputs[i:i + batch_size]
            batch_results = self.predict(batch, **kwargs)
            results.extend(batch_results)

        logger.info(f"Batch inference completed: {len(results)} results")
        return results

    def predict_proba(
        self,
        inputs: Union[str, List[str]]
    ) -> Union[List[float], List[List[float]]]:
        """
        分類タスクの確率を取得

        Args:
            inputs: 入力テキストまたはテキストのリスト

        Returns:
            確率のリスト
        """
        if self.task != "text-classification":
            raise ValueError("predict_probaはtext-classificationタスクでのみ使用可能です")

        results = self.predict(inputs, return_all_scores=True)

        if isinstance(inputs, str):
            # 単一入力
            return [score["score"] for score in results]
        else:
            # 複数入力
            return [[score["score"] for score in result] for result in results]

    def generate_text(
        self,
        prompt: str,
        max_new_tokens: int = 50,
        temperature: float = 0.7,
        num_return_sequences: int = 1,
        **kwargs
    ) -> Union[str, List[str]]:
        """
        テキスト生成

        Args:
            prompt: プロンプト
            max_new_tokens: 生成する新しいトークンの最大数
            temperature: 温度パラメータ
            num_return_sequences: 生成シーケンス数
            **kwargs: その他の引数

        Returns:
            生成されたテキスト
        """
        if self.task != "text-generation":
            raise ValueError("generate_textはtext-generationタスクでのみ使用可能です")

        results = self.predict(
            prompt,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            num_return_sequences=num_return_sequences,
            **kwargs
        )

        if num_return_sequences == 1:
            return results[0]["generated_text"]
        else:
            return [result["generated_text"] for result in results]

## 5. 使用例

In [11]:
# 使用例
from transformers import pipeline

# 感情分析の例
sentiment_analyzer = ModelInference(
    model_path="cardiffnlp/twitter-roberta-base-sentiment-latest",
    task="text-classification"
)

texts = [
    "I love this product!",
    "This is terrible.",
    "It's okay, nothing special."
]

results = sentiment_analyzer.predict(texts)
for text, result in zip(texts, results):
    print(f"Text: {text}")
    print(f"Prediction: {result}")
    print()

# テキスト生成の例
generator = ModelInference(
    model_path="gpt2",
    task="text-generation"
)
prompt = "The future of AI is"
generated = generator.generate_text(prompt, max_new_tokens=50)
print(f"Prompt: {prompt}")
print(f"Generated: {generated}")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return

Text: I love this product!
Prediction: [{'label': 'negative', 'score': 0.004656333941966295}, {'label': 'neutral', 'score': 0.01053917407989502}, {'label': 'positive', 'score': 0.9848045110702515}]

Text: This is terrible.
Prediction: [{'label': 'negative', 'score': 0.9146419763565063}, {'label': 'neutral', 'score': 0.07204394787549973}, {'label': 'positive', 'score': 0.013313977979123592}]

Text: It's okay, nothing special.
Prediction: [{'label': 'negative', 'score': 0.09523523598909378}, {'label': 'neutral', 'score': 0.5986359119415283}, {'label': 'positive', 'score': 0.30612891912460327}]



Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: The future of AI is
Generated: The future of AI is really going to come down to whether you actually like it or not. You start to see what I mean when I say AI is going to become a reality.

Q. What was your reaction when you first announced that you were going to release
